In [1]:
!pip install python-sat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.9 MB/s 


In [2]:
import numpy as np
from pysat.formula import CNF
from itertools import combinations
from pysat.solvers import Solver

# **Number Mind**

In [3]:
# Set the number of digits of the secret sequence
#---------------
n = 5
# CAN BE CHANGED
#---------------

# Create the CNF formula
cnf = CNF()

# Set the secret sequence:
# generate a random sequence of n digits
SecretSeq = np.random.randint(10, size=n)

Problem definition:

$x_{pd} =$ TRUE if in position $p$ there is digit $d$

are boolean variables with two indeces.

In [4]:
# PySAT language uses variables with only one index:
# define translating functions for translating variables
# with two indeces into variables with one index

def Index(p,d):
  # it means that is true that the digit d=0,1,...,9 is
  # in position p=0,1,...,n-1 of the sequence
  return d + p*10 + 1

def PositionDigit(i):
  # the index i=1,...,n*10 means
  # that in the position (ten/decina of the number i-1)
  p = (i-1)//10
  # there is the digit (unit/unità of the number i-1)
  d = (i-1)%10
  return p,d

### **Domain clauses**
Exactly 1 digit in each position:

**At least 1 digit in each position**  (number of possible digits $m=10$)

$\underset{|I|=m-k+1}{\underset{I\subseteq\{0,1,...,m-1\}}{\bigwedge}} \quad \underset{i\in I}{\bigvee} \quad Index(p,i) \quad$ with $k=1 \quad \implies$ $\underset{i\in \{0,1,...,9\}}{\bigvee} \quad Index(p,i)$

For every position $p\in\{0,...,n-1\}$ the clause $Index(p,0)\vee Index(p,1)\vee...\vee Index(p,9)$ is added to the CNF formula.

In [5]:
for p in range(n):
    # add clause to the CNF formula
    cnf.append([Index(p,d) for d in range(10)])

**At most 1 digit in each position** (number of possible digits $m=10$)

$\underset{|I|=k+1}{\underset{I\subseteq\{0,1,...,m-1\}}{\bigwedge}} \quad \underset{i\in I}{\bigvee} \quad \neg\, Index(p,i) \quad$ with $k=1 \quad \implies$ $\underset{|I|=2}{\underset{I\subseteq\{0,1,...,9\}}{\bigwedge}} \quad \underset{i\in I}{\bigvee} \quad \neg\, Index(p,i)$

For every position $p\in\{0,...,n-1\}$ the clauses $\neg(\,Index(p,digit1)\wedge Index(p,digit2)\,)$ are added to the CNF formula, for all $digit1,digit2\in\{0,...,9\}$ such that $digit1\lneqq digit2$.

In [6]:
for p in range(n):
    for d in range(10):
        for d1 in range(d+1,10):
            # add clause to the CNF formula
            cnf.append([-Index(p,d),-Index(p,d1)])

### **Solve**

At every iteration a sequence has to be guessed, considering the feedback on the correct digits of all the previous proposed solutions.

In [7]:
# perform the following steps until the correct sequence is found
while(True):
  
  # feed the solver with the input CNF formula
  solver = Solver(bootstrap_with = cnf.clauses)
  solver.solve()
  # extract a satisfying assignment for the CNF formula given to the solver
  Seq = solver.get_model()
  # translate the sequence of indeces of length n*10 to a sequence of digits of length n
  Seq = [PositionDigit(i)[1] for i in [Seq[j] for j in range(n*10) if Seq[j]>0]]

  # initialize the "number of correct guesses" variable
  k = 0
  # count the number of correct guesses
  for p in range(n):
    if Seq[p] == SecretSeq[p]: k+=1

  # break the while loop if the correct sequence is found
  if k == n: break

  # knowing that exactly k digits of the guessed sequence are in the correct position,
  # all the possible combinations of EXACTLY k correct digits of the guessed sequence
  # (tuples with position and digit) are added as clauses to the CNF formula

  # at least k
  ss = combinations(enumerate(Seq), n - k + 1)
  for s in ss: cnf.append([Index(*pd) for pd in s])
  # at most k
  ss = combinations(enumerate(Seq), k + 1)
  for s in ss: cnf.append([-Index(*pd) for pd in s])

At every iteration, knowing that exactly $k$ digits of the guessed sequence are in the correct position, the following clauses are added to the CNF formula, where $S$ is the set of each possible subsequence of the guessed sequence, sequence written as tuples (position, digit).

**At least $k$ correct digits in the correct position**

$\underset{|s|=n-k+1}{\underset{s\subseteq S}{\bigwedge}} \quad \underset{(p_s,d_s)\in s}{\bigvee} \quad Index(p_s,d_s) \quad$

**At most $k$ correct digits in the correct position**

$\underset{|s|=k+1}{\underset{s\subseteq S}{\bigwedge}} \quad \underset{(p_s,d_s)\in s}{\bigvee} \quad \neg Index(p_s,d_s) \quad$

In [8]:
print("Secret sequence:  ", SecretSeq)
print("Guessed sequence: ", np.array(Seq))

Secret sequence:   [5 3 6 2 9]
Guessed sequence:  [5 3 6 2 9]
